# RAW LAYER

In [1]:
import polars as pl
from datetime import datetime

In [6]:
def load_tables(raw_path: str) -> pl.DataFrame:
    df_raw = pl.read_csv(raw_path)
    return df_raw

In [ ]:
df_customer = load_tables('/home/sapna.choudhary/Data-Engineering-Training/DataWarehouse/DWH_raw_dataset/dim_customer.csv')
df_order = load_tables('/home/sapna.choudhary/Data-Engineering-Training/DataWarehouse/DWH_raw_dataset/dim_order.csv')
df_product = load_tables('/home/sapna.choudhary/Data-Engineering-Training/DataWarehouse/DWH_raw_dataset/dim_product.csv')
df_shipping_type = load_tables('/home/sapna.choudhary/Data-Engineering-Training/DataWarehouse/DWH_raw_dataset/dim_shipping_type.csv')

In [11]:
print(df_customer.shape)
print(df_order.shape)
print(df_product.shape)
print(df_shipping_type.shape)

(500, 15)
(3002, 19)
(101004, 11)
(5, 3)


In [14]:
df_product.head()

product_id,product_name,brand_tier,brand_name,brand_country,main_category,sub_category,discount_percent,actual_price,rating,no of ratings
str,str,str,str,str,str,str,f64,f64,f64,f64
"""prd_00000001""","""Learn Whether (Sell notice him…","""economy""","""HP""","""china""",""" Beauty & PersonAl Care ""","""Ski*ncaRe""",92.881092,144645.8,4.056,7404.0
"""prd_00000002""","""Arm Month (Minute treatment co…","""premium""","""OnePlus""","""china""","""ElectrOnicS""",""" MOBIle* PhOnes """,75.621479,41593.9925,0.0,6885.0
"""prd_00000003""","""Only Pick (Mention should list…",null,"""Adidas""",null,"""Ente*rtAinment""","""MUsi*c CDs""",42.830099,36078.4253,0.688,4340.0
"""prd_00000004""","""From Ago (Through age range.)""",null,"""Canon""","""Japan""","""ACcEso*riEs""",""" OxIDi*sed Jewellery """,-53.996101,57825.174,1.0,null
"""prd_00000005""","""Gas Heart (Thus hard learn.)""","""Standard""","""OnePlus""","""china""","""footweaR""","""Women's Shoes""",-65.465571,42998.885,null,7165.0


In [16]:
df_product.height

101004

In [17]:
df_shipping_type.write_parquet("raw_shipping_type.parquet")

In [19]:
df_shipping_type_parquet = pl.read_parquet("raw_shipping_type.parquet")
df_shipping_type_parquet

shipping_type_id,shipping_type,delivery_estimate
str,str,str
"""ship_0001""","""Express!""","""3-5 days"""
"""ship_0002!""",""" Normal ""","""3-7 days!"""
"""Ship_0003""",""" One-Day Delivery ""","""1 DAY"""
"""Ship_0004""",""" One-Day Delivery ""","""1 DAY"""
"""SHIP_0004""",""" FAST Delivery ""","""1-3 days"""
